In [17]:
# в этом ноутбуке соединяем получившиеся таблички
# и создаём уже нормальный датасет пригодный для работы
import pandas as pd
from tqdm import tqdm
from itertools import cycle
import numpy as np

In [57]:
df_texts = pd.read_parquet("translated_text_table_large_3-8.parquet")

In [3]:
df_frms = pd.read_parquet("WITH_NUM2WORD_augmented_formula_3-8.parquet")

In [58]:
# датафрейм с переведёнными текстами
df_texts.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 926327 entries, 0 to 92631
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   text_id       926327 non-null  int64 
 1   markdown_rus  926327 non-null  object
 2   formulae      926327 non-null  object
 3   tags          926327 non-null  object
dtypes: int64(1), object(3)
memory usage: 35.3+ MB


In [7]:
# датафрейм с аугментированными формулами
df_frms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10092275 entries, 0 to 10092274
Data columns (total 7 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   text_id              int64 
 1   formula_index        int64 
 2   original_formula     object
 3   rand_delet_and_typo  object
 4   synonyms             object
 5   all_augments         object
 6   num2word             object
dtypes: int64(2), object(5)
memory usage: 539.0+ MB


In [8]:
df_frms.sample(2)

,text_id,formula_index,original_formula,rand_delet_and_typo,synonyms,all_augments,num2word
4458339,455611,8,$16$,$16$,$16$,$16$,$шестнадцать$
6677973,695460,0,$X_{n}$,$X_{n}$,$X с индексом {n}$,$X с индексом {n}$,$X_{n}$


In [9]:
df_texts.sample(2)

,text_id,body_markdown,formulae,tags
856916,941067,Recycle deleted answer:\r\n\r\nLet [[0]] be a ...,"[$I_n$, $\mathbb R$, $f_0 = 0$, $f_{n + 1}$, $...","[measure-theory, probability-theory]"
29990,32164,I will use the notation of [[0]] for global di...,"[$\mathcal{D}(M)$, $M$, $\mathbb{R}$, $$\mathc...",[lie-groups]


In [12]:
import random as rand

In [ ]:
def get_repl_forms(forms):
    #print(forms)
    repl_forms = []
    for formula in forms:
        #print(formula)
        repl_forms.append(formula)
    #print(repl_forms)
    return repl_forms

rep_dict={}
ovrl = len(df_texts['text_id'])
cols = cycle(['rand_delet_and_typo', 'synonyms', 'all_augments', 'num2word'])

for text_ind in tqdm(df_texts['text_id']):
    try:
        text_forms = df_frms.loc[df_frms['text_id'] == int(text_ind)]
        col=next(cols)
        #print(col)
        repl_frms = get_repl_forms(text_forms[col])
        rep_dict[text_ind]= (col,repl_frms)
    except Exception as e:
                pass
    finally:
                pass

In [40]:
rep_dict = {}
cols = cycle(['rand_delet_and_typo', 'synonyms', 'all_augments', 'num2word'])

for text_id, group in tqdm(df_frms.groupby('text_id')):
    try:
        col = next(cols)
        repl_frms = group[col].tolist()
        rep_dict[text_id] = (col, repl_frms)
    except Exception as e:
        pass

100%|███████████████████████████████████████████████████████████████████████| 926327/926327 [01:03<00:00, 14602.43it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [42]:
# количество текстов, с которыми что-то пошло не так
len(df_texts)-len(rep_dict)

0

In [43]:
len(rep_dict)

926327

In [47]:
df_texts.sample(2)


,text_id,body_markdown,formulae,tags
742901,811630,This follows from Cauchy-Schwarz inequality by...,"[$x_i=\alpha_i$, $y_i=1$, $i=1,2,...,n$]","[inequality, complex-numbers]"
41547,44550,It is a standard excercise to prove the follow...,"[$(\Omega,\mathcal{F})$, $(\Omega',\mathcal{F}...","[probability-theory, measure-theory]"


In [48]:
import re

In [49]:
# функция для вставки обратно

def insert_frms_back(inpstr : str, replacements : list[str]):
    pattern = r'\[\[\d+\]\]'  # костины скобочки
    # для каждой конкретной формулы в списке формул
    inp_str = inpstr
    for replacement in replacements:
        # надо заменить бэкслеши потому что 
        # предполагаем что формул с больше чем двумя бэкслешами подряд у нас нет
        replacement = replacement.replace('\\', r'\\')
        inp_str = re.sub(pattern,replacement, inp_str, count=1)
    # возвращаем текст со вставленными обратно формулами
    return inp_str

In [ ]:
# меняем replacement list на словарь потому что я дурак
# на будущее - хранить оригинальные формулы (колонку листов) в таблице с текстами 
# rep_dict = {key:value for (key, value) in replacement_list}


In [50]:
orig_frms_dict={}
with open(f"orig_forms_repl_errs.txt", "w") as f:
        for formulae, text_id in tqdm(zip(df_texts['formulae'],df_texts['text_id'])):
            try:
               # f.write(f"\nИндекс: {text_ind}")
               # f.write(f"{df_texts.loc[df_texts['text_id'] == text_ind]}")
                orig_frms_dict[text_id]= formulae
            except Exception as e:
                        f.write(f"\nНеправильный индекс: {text_ind}\nError: {e}")
            finally:
                        pass

926327it [00:01, 826486.09it/s]


In [61]:
len(orig_frms_dict)

926327

In [64]:
len(rep_dict)


926327

In [65]:
len(orig_frms_dict)

926327

In [72]:
df_texts

,text_id,markdown_rus,formulae,tags
0,0,"Для [[0]] и [[1]] таких, что\r\n[[2]]\r\nнерав...","[$0\lt a\lt 1$, $b\gt 0$, $$1-\frac{\lfloor a\...","[inequality, floor-function]"
1,1,Достаточно рассмотреть случай [[0]]. Пусть [[1...,"[$n=2$, $E$, $E_1 = \{x\in E : f_1(x) \ge f_2(...","[real-analysis, measure-theory]"
2,2,"Поскольку [[0]] измеримы, измеримы и [[1]], и\...","[$f_i$, $f_i - f_j$, $L_{ij} = \{x: f_i(x) \le...","[real-analysis, measure-theory]"
3,3,"Нет. Предположим, что [[0]] является индикатор...","[$g$, $[0,1]$, $\|g\|_2 = 1$, $$\int_{\Bbb R} ...","[real-analysis, integration, functional-analysis]"
4,5,Абсолютное значение очень просто:\r\n\r\n[[0]]...,"[$$|z^4|=|-1+\sqrt3 i|=2$$, $$|z|=\sqrt[4]2$$]","[complex-analysis, complex-numbers]"
...,...,...,...,...
92627,1026986,"Еще один, понятный старшеклассникам и ниже.\r\...","[$n$, $n^2$, $n$, $(2k)^2 = 2(2k^2)$, $(2k+1)^...","[elementary-number-theory, proof-writing, radi..."
92628,1026987,"Пусть [[0]] — полиномиальные уравнения, которы...","[$x^2-2=0$, $\pm1,\pm2$, $\pm \sqrt2$]","[elementary-number-theory, proof-writing, radi..."
92629,1026989,Это доказательство основано на происхождении (...,"[$n$, $n^2<D<(n+1)^2$, $\sqrt{D}$, $D=2$, $n=1...","[elementary-number-theory, proof-writing, radi..."
92630,1027011,> *счетно бесконечное* множество — это множест...,"[$a_1,a_2,a_3,...$, $0,1,-1,2,-2,3,-3,...$, $\...","[elementary-set-theory, intuition, infinity, faq]"


In [53]:
from ast import literal_eval
# потому что по какой-то причине всё сохранилось в строках 

In [77]:
rep_dict[1][1]

['$n равняется 2$',
 '$E$',
 '$E с индексом 1 = \\{x входит в  E : f_1(x) \\ge f с индексом 2( хэ  скобка закрывается \\}$',
 '$E с индексом 2  равняется   Е   отнять   и заглавная  с индексом 1$',
 '$\\int_E g  равно   определённый интеграл от { Е _1} f с индексом 1  плюс  \\int_{E с индексом 2}f с индексом 2 \\le \\nu(E с индексом 1 скобки закрываются   плюс  \\nu(E с индексом 2) =  ню ( э прописная  скобки закрываются $']

In [106]:
orig_frms_dict[1][1]

'$E$'

In [109]:
# подставляем испорченные формулы
# в этом же цикле подставляем и чистые формулы тоже
replaced_texts={}
orig_texts={}
tags_dict={}
augment_dict={}
text_ids={}
for text_id, text_markdown, tags in tqdm(zip(df_texts['text_id'], df_texts['markdown_rus'],df_texts['tags'])):
    try:
        augment_type = rep_dict[text_id][0]
        frms_bad = rep_dict[text_id][1]
        
        
        out_str_bad = insert_frms_back(text_markdown, frms_bad)
        
        replaced_texts[text_id] = out_str_bad
        
        frms_orig = orig_frms_dict[text_id]
        out_str_orig = insert_frms_back(text_markdown, frms_orig)
        orig_texts[text_id] = out_str_orig
        tags_dict[text_id] = tags
        augment_dict[text_id] = augment_type
        
    except Exception as e:
        pass
    finally:
        pass
    

926327it [09:13, 1674.24it/s]


In [110]:
len(replaced_texts)


926327

In [111]:
len(orig_texts)


926327

In [143]:
orig_texts.keys

<function dict.keys>

In [112]:
# по-хорошему - надо найти пересечение их ключей
same_keys = set.intersection(set(replaced_texts.keys()),set(orig_texts.keys()))

In [115]:
# всё в порядке, 
len(same_keys) == len(orig_texts) == len(replaced_texts)

True

In [ ]:
orig_texts.values()

In [117]:
targets = list(orig_texts.values())
inputs = list(replaced_texts.values())
tags = list(tags_dict.values())
augmentations = list(augment_dict.values())

In [145]:
df_dict={
    'text_id' : orig_texts.keys(),
    'output_texts': targets,
    'input_texts': inputs,
    'tags' : tags,
    'augmentation' : augmentations,
}

In [146]:
df=pd.DataFrame.from_dict(df_dict,orient='index').transpose()

In [147]:
# наш почти финальный датафрейм
df

,text_ids,output_texts,input_texts,tags,augmentation
0,0,"Для $0\lt a\lt 1$ и $b\gt 0$ таких, что\r\n$$1...","Для $0 lt a lt 1$ и $b\gt 0$ таких, что\r\n$$1...","[inequality, floor-function]",rand_delet_and_typo
1,1,Достаточно рассмотреть случай $n=2$. Пусть $E$...,Достаточно рассмотреть случай $n равняется 2$....,"[real-analysis, measure-theory]",synonyms
2,2,"Поскольку $f_i$ измеримы, измеримы и $f_i - f_...","Поскольку $ фэ _i$ измеримы, измеримы и $ эф _...","[real-analysis, measure-theory]",all_augments
3,3,"Нет. Предположим, что $g$ является индикатором...","Нет. Предположим, что $g$ является индикатором...","[real-analysis, integration, functional-analysis]",num2word
4,5,Абсолютное значение очень просто:\r\n\r\n$$|z^...,Абсолютное значение очень просто:\r\n\r\n$$|z^...,"[complex-analysis, complex-numbers]",rand_delet_and_typo
...,...,...,...,...,...
926322,1026986,"Еще один, понятный старшеклассникам и ниже.\r\...","Еще один, понятный старшеклассникам и ниже.\r\...","[elementary-number-theory, proof-writing, radi...",all_augments
926323,1026987,"Пусть $x^2-2=0$ — полиномиальные уравнения, ко...",Пусть $x^дваминус два=нулевой$ — полиномиальны...,"[elementary-number-theory, proof-writing, radi...",num2word
926324,1026989,Это доказательство основано на происхождении (...,Это доказательство основано на происхождении (...,"[elementary-number-theory, proof-writing, radi...",rand_delet_and_typo
926325,1027011,> *счетно бесконечное* множество — это множест...,> *счетно бесконечное* множество — это множест...,"[elementary-set-theory, intuition, infinity, faq]",synonyms


In [105]:
# сохраним его на всякий случай
df.to_parquet("TEXTS_ONLY_DF_FROM_3-8.parquet")

In [123]:
unique_frms = df_frms[~df_frms['original_formula'].duplicated(keep=False)]

In [125]:
unique_frms

,text_id,formula_index,original_formula,rand_delet_and_typo,synonyms,all_augments,num2word
2,0,2,"$$1-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}{N}...","$$1-\frac{\lflooor a\cdot\min\{N,K\}\rfloor}{N...",$$1 отнять \frac{\lfloor эй \cdot\min\{ эн бо...,$$1 оттнять \frac{\lfloor эй \cdot\min\{{ эн ...,"$$один-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}..."
3,0,3,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...","-\frac{(\lfloor a\cdot\mathrm{min \{N,K\ \rf...",$$-\frac{ в круглых скобках \lfloor а умножи...,$$-\frac{ в круглых скобках \lfloor а умножи...,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf..."
5,0,5,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...,$$\begin{align}(1 скобки закрываются &\iff -\f...,$$\begin{align}(1 скобки закрываются &\iff -\f...,$$\begin{align}(один)&\iff -\frac{(\lfloor a\c...
7,0,7,$(N-\lfloor aN\rfloor)(N-aN)\gt 0$,$( N-\lfloor aN\rfloor)(N-aN)\gt 0$,$( эн большая отнять \lfloor aN\rfloor скобки...,$( эн большая отнять \lfloor aN\rfloor ск обк...,$(N-\lfloor aN\rfloor)(N-aN)\gt ноль$
8,0,8,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...,$$\begin{align} скобки открываются 2)&\iff (N-...,$$\begiL{align} скобки открыжааются 2)&\iff (N...,$$\begin{align}(два)&\iff (N-aN)\lfloor aN\rfl...
...,...,...,...,...,...,...,...
10092241,1027011,3,"$\frac{1}{1}, \frac{2}{2}$","$\frac{1}{1}, \frac{2}{2}$","$1 разделить на 1, дробь от 2 делить на 2$","$1 разделить на 1, дробь от 2 делить на 2$","$\frac{один}{первый}, \frac{два}{второй}$"
10092242,1027011,4,"$\frac{1}{2}, \frac{2}{4}$","$\frac{1}{2}, \frac{2}{4}$","$дробь от 1 делить на 2, 2 разделить на 4$","$дробь от 1 делить на 2, 2 разделить на 4$","$\frac{один}{второй}, \frac{два}{четвёртый}$"
10092243,1027011,5,"$\frac{1}{1},\frac{1}{2},\frac{2}{1},\frac{1}{...","\frac{1}{1},\frac{1}{{2},\frac{2}}{1},\frac{1}...","$1 делить на 1,1 делить на 2,дробь от 2 делить...","$1 делить на 1,1 делить на 2,дрообь от 2 делит...","$\frac{один}{первый},\frac{один}{второй},\frac..."
10092246,1027011,8,"$0,q_1,-q_1,q_2,-q_2,q_3,-q_3,...$","$0,q_1Ю-q_1,q_2,-qэ2,q_3,-q_3,...$","$0,q_1,- ку с индексом 1, ку _2,- кью с инде...","$0,q_ 1,- ку с индексом 1, ку _2,- кьж с инд...","$ноль,q_первый,-q_один,q_второй,-q_два,q_трети..."


In [135]:
unique_frms.to_parquet("UNIQUE_FORMULAS_FROM_3-8.parquet")

In [ ]:
unique_frms['formulae_only']=True

In [ ]:
unique_frms = pd.read_parquet("UNIQUE_FORMULAS_FROM_3-8.parquet")

In [9]:
df = pd.read_parquet("TEXTS_ONLY_DF_FROM_3-8.parquet")

In [7]:
unique_frms.head(2)

,text_id,formula_index,original_formula,rand_delet_and_typo,synonyms,all_augments,num2word,formulae_only
2,0,2,"$$1-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}{N}...","$$1-\frac{\lflooor a\cdot\min\{N,K\}\rfloor}{N...",$$1 отнять \frac{\lfloor эй \cdot\min\{ эн бо...,$$1 оттнять \frac{\lfloor эй \cdot\min\{{ эн ...,"$$один-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}...",True
3,0,3,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...","-\frac{(\lfloor a\cdot\mathrm{min \{N,K\ \rf...",$$-\frac{ в круглых скобках \lfloor а умножи...,$$-\frac{ в круглых скобках \lfloor а умножи...,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...",True


In [14]:
df.head(2)

,text_id,output_texts,input_texts,tags,augmentation
0,0,"Для $0\lt a\lt 1$ и $b\gt 0$ таких, что\r\n$$1...","Для $0 lt a lt 1$ и $b\gt 0$ таких, что\r\n$$1...","[inequality, floor-function]",rand_delet_and_typo
1,1,Достаточно рассмотреть случай $n=2$. Пусть $E$...,Достаточно рассмотреть случай $n равняется 2$....,"[real-analysis, measure-theory]",synonyms


In [15]:
# добавляем формулам теги
unique_frms_with_tags = pd.merge(unique_frms, df, on='text_id', how='left')

In [ ]:
# удаляем ненужные колонки
unique_frms_with_tags = unique_frms_with_tags.drop(columns=['formula_index','output_texts','input_texts'])

In [40]:
augments = ['rand_delet_and_typo', 'synonyms', 'all_augments', 'num2word']

# выбираем чем будем аугментировать
unique_frms_with_tags['augmentation'] = np.random.choice(augments, size=len(unique_frms_with_tags))

In [107]:
unique_frms_with_tags

,text_id,original_formula,rand_delet_and_typo,synonyms,all_augments,num2word,formulae_only,tags,augmentation,input_texts
0,0,"$$1-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}{N}...","$$1-\frac{\lflooor a\cdot\min\{N,K\}\rfloor}{N...",$$1 отнять \frac{\lfloor эй \cdot\min\{ эн бо...,$$1 оттнять \frac{\lfloor эй \cdot\min\{{ эн ...,"$$один-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}...",True,"[inequality, floor-function]",synonyms,$$1 отнять \frac{\lfloor эй \cdot\min\{ эн бо...
1,0,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...","-\frac{(\lfloor a\cdot\mathrm{min \{N,K\ \rf...",$$-\frac{ в круглых скобках \lfloor а умножи...,$$-\frac{ в круглых скобках \lfloor а умножи...,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...",True,"[inequality, floor-function]",rand_delet_and_typo,"-\frac{(\lfloor a\cdot\mathrm{min \{N,K\ \rf..."
2,0,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...,$$\begin{align}(1 скобки закрываются &\iff -\f...,$$\begin{align}(1 скобки закрываются &\iff -\f...,$$\begin{align}(один)&\iff -\frac{(\lfloor a\c...,True,"[inequality, floor-function]",num2word,$$\begin{align}(один)&\iff -\frac{(\lfloor a\c...
3,0,$(N-\lfloor aN\rfloor)(N-aN)\gt 0$,$( N-\lfloor aN\rfloor)(N-aN)\gt 0$,$( эн большая отнять \lfloor aN\rfloor скобки...,$( эн большая отнять \lfloor aN\rfloor ск обк...,$(N-\lfloor aN\rfloor)(N-aN)\gt ноль$,True,"[inequality, floor-function]",all_augments,$( эн большая отнять \lfloor aN\rfloor ск обк...
4,0,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...,$$\begin{align} скобки открываются 2)&\iff (N-...,$$\begiL{align} скобки открыжааются 2)&\iff (N...,$$\begin{align}(два)&\iff (N-aN)\lfloor aN\rfl...,True,"[inequality, floor-function]",synonyms,$$\begin{align} скобки открываются 2)&\iff (N-...
...,...,...,...,...,...,...,...,...,...,...
3944996,1027011,"$\frac{1}{1}, \frac{2}{2}$","$\frac{1}{1}, \frac{2}{2}$","$1 разделить на 1, дробь от 2 делить на 2$","$1 разделить на 1, дробь от 2 делить на 2$","$\frac{один}{первый}, \frac{два}{второй}$",True,"[elementary-set-theory, intuition, infinity, faq]",all_augments,"$1 разделить на 1, дробь от 2 делить на 2$"
3944997,1027011,"$\frac{1}{2}, \frac{2}{4}$","$\frac{1}{2}, \frac{2}{4}$","$дробь от 1 делить на 2, 2 разделить на 4$","$дробь от 1 делить на 2, 2 разделить на 4$","$\frac{один}{второй}, \frac{два}{четвёртый}$",True,"[elementary-set-theory, intuition, infinity, faq]",synonyms,"$дробь от 1 делить на 2, 2 разделить на 4$"
3944998,1027011,"$\frac{1}{1},\frac{1}{2},\frac{2}{1},\frac{1}{...","\frac{1}{1},\frac{1}{{2},\frac{2}}{1},\frac{1}...","$1 делить на 1,1 делить на 2,дробь от 2 делить...","$1 делить на 1,1 делить на 2,дрообь от 2 делит...","$\frac{один}{первый},\frac{один}{второй},\frac...",True,"[elementary-set-theory, intuition, infinity, faq]",synonyms,"$1 делить на 1,1 делить на 2,дробь от 2 делить..."
3944999,1027011,"$0,q_1,-q_1,q_2,-q_2,q_3,-q_3,...$","$0,q_1Ю-q_1,q_2,-qэ2,q_3,-q_3,...$","$0,q_1,- ку с индексом 1, ку _2,- кью с инде...","$0,q_ 1,- ку с индексом 1, ку _2,- кьж с инд...","$ноль,q_первый,-q_один,q_второй,-q_два,q_трети...",True,"[elementary-set-theory, intuition, infinity, faq]",num2word,"$ноль,q_первый,-q_один,q_второй,-q_два,q_трети..."


In [50]:
# функция для выбора аугментации формул построчно
def aug_row(row):
    return row[row['augmentation']]

In [51]:
# создаём колонку в которую записываем соответствующую формулу
unique_frms_with_tags['input_texts'] = unique_frms_with_tags.apply(aug_row, axis=1)

In [55]:
unique_frms_with_tags.sample(3)

,text_id,original_formula,rand_delet_and_typo,synonyms,all_augments,num2word,formulae_only,tags,augmentation,input_texts
3534997,919253,$$\r\n [y]^m [x]^n \frac{-\log(1-x)}{1-x} \c...,$$\r\n [y]^m [x]гn \frac{-\log(1-x)} {1-x} \...,$$\r\n [y]^m [ х ]^ эн -\log(1-x скобки зак...,$$\r\n [y]^m [ х ]^ эн -\log(1-xскоббки зак...,$$\r\n [y]^m [x]^n \frac{-\log(один-x)}{перв...,True,[combinatorics],num2word,$$\r\n [y]^m [x]^n \frac{-\log(один-x)}{перв...
3700715,959601,$f: \mathbb N \to \mathbb N^4$,$f: \mathbb N \to \mathbb N^4$,$ фэ : \mathbb N \rightarrow \mathbb N^4$,$ фэ : \mathbb N \rightarrow \mathbb N^4$,$f: \mathbb N \to \mathbb N^четыре$,True,[real-analysis],all_augments,$ фэ : \mathbb N \rightarrow \mathbb N^4$
3878126,1005097,$$f(z) = \frac{1}{2\pi i} \int_C \frac{f(\zeta...,$$f(z) = frac{1}{2 pi i} int_C frac{f( zeta...,$$ фэ ( зет скобка закрывается = дробь от 1 ...,$$ фэ ( зет скобказакрываается = дробь от 1 ...,$$f(z) = \frac{один}{второй\pi i} \int_C \frac...,True,[complex-analysis],synonyms,$$ фэ ( зет скобка закрывается = дробь от 1 ...


In [56]:
# нам уже не надо следить за конкретными аугментациями, т.ч. их можем дропнуть
tagged_unique_frms=unique_frms_with_tags.drop(columns=['rand_delet_and_typo', 'synonyms','all_augments','num2word'])

In [57]:
tagged_unique_frms

,text_id,original_formula,formulae_only,tags,augmentation,input_texts
0,0,"$$1-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}{N}...",True,"[inequality, floor-function]",synonyms,$$1 отнять \frac{\lfloor эй \cdot\min\{ эн бо...
1,0,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...",True,"[inequality, floor-function]",rand_delet_and_typo,"-\frac{(\lfloor a\cdot\mathrm{min \{N,K\ \rf..."
2,0,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...,True,"[inequality, floor-function]",num2word,$$\begin{align}(один)&\iff -\frac{(\lfloor a\c...
3,0,$(N-\lfloor aN\rfloor)(N-aN)\gt 0$,True,"[inequality, floor-function]",all_augments,$( эн большая отнять \lfloor aN\rfloor ск обк...
4,0,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...,True,"[inequality, floor-function]",synonyms,$$\begin{align} скобки открываются 2)&\iff (N-...
...,...,...,...,...,...,...
3944996,1027011,"$\frac{1}{1}, \frac{2}{2}$",True,"[elementary-set-theory, intuition, infinity, faq]",all_augments,"$1 разделить на 1, дробь от 2 делить на 2$"
3944997,1027011,"$\frac{1}{2}, \frac{2}{4}$",True,"[elementary-set-theory, intuition, infinity, faq]",synonyms,"$дробь от 1 делить на 2, 2 разделить на 4$"
3944998,1027011,"$\frac{1}{1},\frac{1}{2},\frac{2}{1},\frac{1}{...",True,"[elementary-set-theory, intuition, infinity, faq]",synonyms,"$1 делить на 1,1 делить на 2,дробь от 2 делить..."
3944999,1027011,"$0,q_1,-q_1,q_2,-q_2,q_3,-q_3,...$",True,"[elementary-set-theory, intuition, infinity, faq]",num2word,"$ноль,q_первый,-q_один,q_второй,-q_два,q_трети..."


In [ ]:
# а теперь допустим мы хотим создать датасет
# где первые N экземпляров -- формулы вне текстов
# а вторые N -- тексты с формулами
# но нам т.ж. нужно дать этим формулам правильные теги...

In [59]:
# убираем там где у нас ничего не поменялось
tg_unique_frms = tagged_unique_frms[tagged_unique_frms['original_formula'] != tagged_unique_frms['input_texts']]

In [200]:
# итак
# у нас есть датафрейм с выборкой из формул, отметкой текста, к которой формула принадлежит
# с тегами, отметкой того что это формулы, способом их аугментации
# и самой аугментацией
# все они -- уникальные
# и более того -- все они Точно аугментированные
tg_unique_frms.head(650008)

,text_id,original_formula,formulae_only,tags,augmentation,input_texts
0,0,"$$1-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}{N}...",True,"[inequality, floor-function]",synonyms,$$1 отнять \frac{\lfloor эй \cdot\min\{ эн бо...
1,0,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...",True,"[inequality, floor-function]",rand_delet_and_typo,"-\frac{(\lfloor a\cdot\mathrm{min \{N,K\ \rf..."
2,0,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...,True,"[inequality, floor-function]",num2word,$$\begin{align}(один)&\iff -\frac{(\lfloor a\c...
3,0,$(N-\lfloor aN\rfloor)(N-aN)\gt 0$,True,"[inequality, floor-function]",all_augments,$( эн большая отнять \lfloor aN\rfloor ск обк...
4,0,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...,True,"[inequality, floor-function]",synonyms,$$\begin{align} скобки открываются 2)&\iff (N-...
...,...,...,...,...,...,...
814599,200162,$\sin(\pi n) = 0$,True,"[integration, summation, fourier-series]",all_augments,$ синус скобка открывается пи n скобки закр...
814600,200162,$$\sum_{n = -\infty}^\infty \left\lvert \frac{...,True,"[integration, summation, fourier-series]",rand_delet_and_typo,$$\sum_{n = -\infty}^\infty \left\lvert \frac{...
814601,200162,$\frac{1}{2\pi}\int_{-\pi}^\pi |e^{-i\sigma x}...,True,"[integration, summation, fourier-series]",num2word,$\frac{один}{второй\pi}\int_{-\pi}^\pi |e^{-i\...
814602,200162,$$1 = \frac{\sin^2(\pi \sigma)}{\pi^2} \sum_{n...,True,"[integration, summation, fourier-series]",rand_delet_and_typo,$$1 = \frac{\sinс2(\pi \sigma)}{\pi^2} \sum_{n...


In [62]:
tg_unique_frms.to_parquet("unique_forms_3-8_WITH_METADATA_for_df_formation.parquet")

In [201]:
# берём первые таких хороших уникальных формул
# далее -- создание под каждую конкретную ситуацию
unique_frms_short = tg_unique_frms.head(650008)

In [202]:
unique_frms_short.head(5)

,text_id,original_formula,formulae_only,tags,augmentation,input_texts
0,0,"$$1-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}{N}...",True,"[inequality, floor-function]",synonyms,$$1 отнять \frac{\lfloor эй \cdot\min\{ эн бо...
1,0,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...",True,"[inequality, floor-function]",rand_delet_and_typo,"-\frac{(\lfloor a\cdot\mathrm{min \{N,K\ \rf..."
2,0,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...,True,"[inequality, floor-function]",num2word,$$\begin{align}(один)&\iff -\frac{(\lfloor a\c...
3,0,$(N-\lfloor aN\rfloor)(N-aN)\gt 0$,True,"[inequality, floor-function]",all_augments,$( эн большая отнять \lfloor aN\rfloor ск обк...
4,0,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...,True,"[inequality, floor-function]",synonyms,$$\begin{align} скобки открываются 2)&\iff (N-...


In [203]:
unique_frms_short.rename(columns={'original_formula':'output_texts'}, inplace=True)

C:\Users\blody\AppData\Local\Temp\ipykernel_34152\894079066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_frms_short.rename(columns={'original_formula':'output_texts'}, inplace=True)


In [204]:
unique_frms_short

,text_id,output_texts,formulae_only,tags,augmentation,input_texts
0,0,"$$1-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}{N}...",True,"[inequality, floor-function]",synonyms,$$1 отнять \frac{\lfloor эй \cdot\min\{ эн бо...
1,0,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...",True,"[inequality, floor-function]",rand_delet_and_typo,"-\frac{(\lfloor a\cdot\mathrm{min \{N,K\ \rf..."
2,0,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...,True,"[inequality, floor-function]",num2word,$$\begin{align}(один)&\iff -\frac{(\lfloor a\c...
3,0,$(N-\lfloor aN\rfloor)(N-aN)\gt 0$,True,"[inequality, floor-function]",all_augments,$( эн большая отнять \lfloor aN\rfloor ск обк...
4,0,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...,True,"[inequality, floor-function]",synonyms,$$\begin{align} скобки открываются 2)&\iff (N-...
...,...,...,...,...,...,...
814599,200162,$\sin(\pi n) = 0$,True,"[integration, summation, fourier-series]",all_augments,$ синус скобка открывается пи n скобки закр...
814600,200162,$$\sum_{n = -\infty}^\infty \left\lvert \frac{...,True,"[integration, summation, fourier-series]",rand_delet_and_typo,$$\sum_{n = -\infty}^\infty \left\lvert \frac{...
814601,200162,$\frac{1}{2\pi}\int_{-\pi}^\pi |e^{-i\sigma x}...,True,"[integration, summation, fourier-series]",num2word,$\frac{один}{второй\pi}\int_{-\pi}^\pi |e^{-i\...
814602,200162,$$1 = \frac{\sin^2(\pi \sigma)}{\pi^2} \sum_{n...,True,"[integration, summation, fourier-series]",rand_delet_and_typo,$$1 = \frac{\sinс2(\pi \sigma)}{\pi^2} \sum_{n...


In [191]:
df = df[df['output_texts'] != df['input_texts']]

In [193]:
df

,text_id,output_texts,input_texts,tags,augmentation,formulae_only
0,0,"Для $0\lt a\lt 1$ и $b\gt 0$ таких, что\r\n$$1...","Для $0 lt a lt 1$ и $b\gt 0$ таких, что\r\n$$1...","[inequality, floor-function]",rand_delet_and_typo,False
1,1,Достаточно рассмотреть случай $n=2$. Пусть $E$...,Достаточно рассмотреть случай $n равняется 2$....,"[real-analysis, measure-theory]",synonyms,False
2,2,"Поскольку $f_i$ измеримы, измеримы и $f_i - f_...","Поскольку $ фэ _i$ измеримы, измеримы и $ эф _...","[real-analysis, measure-theory]",all_augments,False
3,3,"Нет. Предположим, что $g$ является индикатором...","Нет. Предположим, что $g$ является индикатором...","[real-analysis, integration, functional-analysis]",num2word,False
4,5,Абсолютное значение очень просто:\r\n\r\n$$|z^...,Абсолютное значение очень просто:\r\n\r\n$$|z^...,"[complex-analysis, complex-numbers]",rand_delet_and_typo,False
...,...,...,...,...,...,...
926322,1026986,"Еще один, понятный старшеклассникам и ниже.\r\...","Еще один, понятный старшеклассникам и ниже.\r\...","[elementary-number-theory, proof-writing, radi...",all_augments,False
926323,1026987,"Пусть $x^2-2=0$ — полиномиальные уравнения, ко...",Пусть $x^дваминус два=нулевой$ — полиномиальны...,"[elementary-number-theory, proof-writing, radi...",num2word,False
926324,1026989,Это доказательство основано на происхождении (...,Это доказательство основано на происхождении (...,"[elementary-number-theory, proof-writing, radi...",rand_delet_and_typo,False
926325,1027011,> *счетно бесконечное* множество — это множест...,> *счетно бесконечное* множество — это множест...,"[elementary-set-theory, intuition, infinity, faq]",synonyms,False


In [205]:
# т.к. уникальные формулы у нас закончились на 200162 индексе, примерно, то тут с запасом берём то что > по индексу
# убираем text_id и 
short_df = df[df.index>200162]

In [206]:
# текстов получается в два раза больше чем формул
short_df

,text_id,output_texts,input_texts,tags,augmentation,formulae_only
200163,215030,Я бы попробовал так: поскольку $ab > 0$ сущест...,Я бы попробовал так: поскольку $ab > ноль$ сущ...,[algebra-precalculus],num2word,False
200164,215031,"Как насчет $f(x)\equiv1$ для первого, с тех по...","Как насчет $f(x)\equiv1$ для первого, с тех по...","[real-analysis, measure-theory, lebesgue-integ...",rand_delet_and_typo,False
200165,215032,"По определению, функция интегрируема по Лебегу...","По определению, функция интегрируема по Лебегу...","[real-analysis, measure-theory, lebesgue-integ...",synonyms,False
200166,215034,"Я сделаю все возможное, чтобы донести до вас э...","Я сделаю все возможное, чтобы донести до вас э...","[taylor-expansion, approximation]",all_augments,False
200167,215035,**Подсказка:** используя тождество $1-\cos^2x=...,**Подсказка:** используя тождество $один-\cos^...,"[calculus, limits, trigonometry, limits-withou...",num2word,False
...,...,...,...,...,...,...
926322,1026986,"Еще один, понятный старшеклассникам и ниже.\r\...","Еще один, понятный старшеклассникам и ниже.\r\...","[elementary-number-theory, proof-writing, radi...",all_augments,False
926323,1026987,"Пусть $x^2-2=0$ — полиномиальные уравнения, ко...",Пусть $x^дваминус два=нулевой$ — полиномиальны...,"[elementary-number-theory, proof-writing, radi...",num2word,False
926324,1026989,Это доказательство основано на происхождении (...,Это доказательство основано на происхождении (...,"[elementary-number-theory, proof-writing, radi...",rand_delet_and_typo,False
926325,1027011,> *счетно бесконечное* множество — это множест...,> *счетно бесконечное* множество — это множест...,"[elementary-set-theory, intuition, infinity, faq]",synonyms,False


In [207]:
# в предыдущей ячейке мы выбрали только 300к таких текстов
# чтобы в итоге было по-честному с формулами
short_df.tail()

,text_id,output_texts,input_texts,tags,augmentation,formulae_only
926322,1026986,"Еще один, понятный старшеклассникам и ниже.\r\...","Еще один, понятный старшеклассникам и ниже.\r\...","[elementary-number-theory, proof-writing, radi...",all_augments,False
926323,1026987,"Пусть $x^2-2=0$ — полиномиальные уравнения, ко...",Пусть $x^дваминус два=нулевой$ — полиномиальны...,"[elementary-number-theory, proof-writing, radi...",num2word,False
926324,1026989,Это доказательство основано на происхождении (...,Это доказательство основано на происхождении (...,"[elementary-number-theory, proof-writing, radi...",rand_delet_and_typo,False
926325,1027011,> *счетно бесконечное* множество — это множест...,> *счетно бесконечное* множество — это множест...,"[elementary-set-theory, intuition, infinity, faq]",synonyms,False
926326,1027014,> Как могут быть разные виды бесконечностей?\r...,> Как могут быть разные виды бесконечностей?\r...,"[elementary-set-theory, intuition, infinity, faq]",all_augments,False


In [97]:
short_df.drop(columns='text_id', inplace=True)

In [208]:
# добавляем колонку с метаданными по принадлежности к текстам
short_df['formulae_only']=False

C:\Users\blody\AppData\Local\Temp\ipykernel_34152\3265594950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_df['formulae_only']=False


In [209]:
short_df.head()

,text_id,output_texts,input_texts,tags,augmentation,formulae_only
200163,215030,Я бы попробовал так: поскольку $ab > 0$ сущест...,Я бы попробовал так: поскольку $ab > ноль$ сущ...,[algebra-precalculus],num2word,False
200164,215031,"Как насчет $f(x)\equiv1$ для первого, с тех по...","Как насчет $f(x)\equiv1$ для первого, с тех по...","[real-analysis, measure-theory, lebesgue-integ...",rand_delet_and_typo,False
200165,215032,"По определению, функция интегрируема по Лебегу...","По определению, функция интегрируема по Лебегу...","[real-analysis, measure-theory, lebesgue-integ...",synonyms,False
200166,215034,"Я сделаю все возможное, чтобы донести до вас э...","Я сделаю все возможное, чтобы донести до вас э...","[taylor-expansion, approximation]",all_augments,False
200167,215035,**Подсказка:** используя тождество $1-\cos^2x=...,**Подсказка:** используя тождество $один-\cos^...,"[calculus, limits, trigonometry, limits-withou...",num2word,False


In [212]:
unique_frms_short.head()

,text_id,output_texts,formulae_only,tags,augmentation,input_texts
0,0,"$$1-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}{N}...",True,"[inequality, floor-function]",synonyms,$$1 отнять \frac{\lfloor эй \cdot\min\{ эн бо...
1,0,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...",True,"[inequality, floor-function]",rand_delet_and_typo,"-\frac{(\lfloor a\cdot\mathrm{min \{N,K\ \rf..."
2,0,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...,True,"[inequality, floor-function]",num2word,$$\begin{align}(один)&\iff -\frac{(\lfloor a\c...
3,0,$(N-\lfloor aN\rfloor)(N-aN)\gt 0$,True,"[inequality, floor-function]",all_augments,$( эн большая отнять \lfloor aN\rfloor ск обк...
4,0,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...,True,"[inequality, floor-function]",synonyms,$$\begin{align} скобки открываются 2)&\iff (N-...


In [85]:
# для формул убираем колонку text_id
# и переставляем колонки местами
# решил в итоге индексы текстов не убирать на всякий случай

In [213]:
un_frms_short=unique_frms_short[['text_id','output_texts','input_texts',
                            'tags','augmentation','formulae_only']]

In [217]:
# смотрим что всё в порядке с формулами
un_frms_short

,text_id,output_texts,input_texts,tags,augmentation,formulae_only
0,0,"$$1-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}{N}...",$$1 отнять \frac{\lfloor эй \cdot\min\{ эн бо...,"[inequality, floor-function]",synonyms,True
1,0,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf...","-\frac{(\lfloor a\cdot\mathrm{min \{N,K\ \rf...","[inequality, floor-function]",rand_delet_and_typo,True
2,0,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...,$$\begin{align}(один)&\iff -\frac{(\lfloor a\c...,"[inequality, floor-function]",num2word,True
3,0,$(N-\lfloor aN\rfloor)(N-aN)\gt 0$,$( эн большая отнять \lfloor aN\rfloor ск обк...,"[inequality, floor-function]",all_augments,True
4,0,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...,$$\begin{align} скобки открываются 2)&\iff (N-...,"[inequality, floor-function]",synonyms,True
...,...,...,...,...,...,...
814599,200162,$\sin(\pi n) = 0$,$ синус скобка открывается пи n скобки закр...,"[integration, summation, fourier-series]",all_augments,True
814600,200162,$$\sum_{n = -\infty}^\infty \left\lvert \frac{...,$$\sum_{n = -\infty}^\infty \left\lvert \frac{...,"[integration, summation, fourier-series]",rand_delet_and_typo,True
814601,200162,$\frac{1}{2\pi}\int_{-\pi}^\pi |e^{-i\sigma x}...,$\frac{один}{второй\pi}\int_{-\pi}^\pi |e^{-i\...,"[integration, summation, fourier-series]",num2word,True
814602,200162,$$1 = \frac{\sin^2(\pi \sigma)}{\pi^2} \sum_{n...,$$1 = \frac{\sinс2(\pi \sigma)}{\pi^2} \sum_{n...,"[integration, summation, fourier-series]",rand_delet_and_typo,True


In [218]:
# смотрим что всё в порядке с текстами
short_df

,text_id,output_texts,input_texts,tags,augmentation,formulae_only
200163,215030,Я бы попробовал так: поскольку $ab > 0$ сущест...,Я бы попробовал так: поскольку $ab > ноль$ сущ...,[algebra-precalculus],num2word,False
200164,215031,"Как насчет $f(x)\equiv1$ для первого, с тех по...","Как насчет $f(x)\equiv1$ для первого, с тех по...","[real-analysis, measure-theory, lebesgue-integ...",rand_delet_and_typo,False
200165,215032,"По определению, функция интегрируема по Лебегу...","По определению, функция интегрируема по Лебегу...","[real-analysis, measure-theory, lebesgue-integ...",synonyms,False
200166,215034,"Я сделаю все возможное, чтобы донести до вас э...","Я сделаю все возможное, чтобы донести до вас э...","[taylor-expansion, approximation]",all_augments,False
200167,215035,**Подсказка:** используя тождество $1-\cos^2x=...,**Подсказка:** используя тождество $один-\cos^...,"[calculus, limits, trigonometry, limits-withou...",num2word,False
...,...,...,...,...,...,...
926322,1026986,"Еще один, понятный старшеклассникам и ниже.\r\...","Еще один, понятный старшеклассникам и ниже.\r\...","[elementary-number-theory, proof-writing, radi...",all_augments,False
926323,1026987,"Пусть $x^2-2=0$ — полиномиальные уравнения, ко...",Пусть $x^дваминус два=нулевой$ — полиномиальны...,"[elementary-number-theory, proof-writing, radi...",num2word,False
926324,1026989,Это доказательство основано на происхождении (...,Это доказательство основано на происхождении (...,"[elementary-number-theory, proof-writing, radi...",rand_delet_and_typo,False
926325,1027011,> *счетно бесконечное* множество — это множест...,> *счетно бесконечное* множество — это множест...,"[elementary-set-theory, intuition, infinity, faq]",synonyms,False


In [246]:
df_finfin = pd.concat([short_df, un_frms_short], ignore_index=True)

In [247]:
df_finfin

,text_id,output_texts,input_texts,tags,augmentation,formulae_only
0,215030,Я бы попробовал так: поскольку $ab > 0$ сущест...,Я бы попробовал так: поскольку $ab > ноль$ сущ...,[algebra-precalculus],num2word,False
1,215031,"Как насчет $f(x)\equiv1$ для первого, с тех по...","Как насчет $f(x)\equiv1$ для первого, с тех по...","[real-analysis, measure-theory, lebesgue-integ...",rand_delet_and_typo,False
2,215032,"По определению, функция интегрируема по Лебегу...","По определению, функция интегрируема по Лебегу...","[real-analysis, measure-theory, lebesgue-integ...",synonyms,False
3,215034,"Я сделаю все возможное, чтобы донести до вас э...","Я сделаю все возможное, чтобы донести до вас э...","[taylor-expansion, approximation]",all_augments,False
4,215035,**Подсказка:** используя тождество $1-\cos^2x=...,**Подсказка:** используя тождество $один-\cos^...,"[calculus, limits, trigonometry, limits-withou...",num2word,False
...,...,...,...,...,...,...
1314988,200162,$\sin(\pi n) = 0$,$ синус скобка открывается пи n скобки закр...,"[integration, summation, fourier-series]",all_augments,True
1314989,200162,$$\sum_{n = -\infty}^\infty \left\lvert \frac{...,$$\sum_{n = -\infty}^\infty \left\lvert \frac{...,"[integration, summation, fourier-series]",rand_delet_and_typo,True
1314990,200162,$\frac{1}{2\pi}\int_{-\pi}^\pi |e^{-i\sigma x}...,$\frac{один}{второй\pi}\int_{-\pi}^\pi |e^{-i\...,"[integration, summation, fourier-series]",num2word,True
1314991,200162,$$1 = \frac{\sin^2(\pi \sigma)}{\pi^2} \sum_{n...,$$1 = \frac{\sinс2(\pi \sigma)}{\pi^2} \sum_{n...,"[integration, summation, fourier-series]",rand_delet_and_typo,True


In [227]:
df_finfin.to_parquet("LARGE_DF_WITH_METADATA_3-8_UNSHUFFLED.parquet", index=False)

In [248]:
df_finfin = df_finfin.sample(frac=1, random_state=42)

In [233]:
# 15000 записей сохраняем исключительно в целях тестирования 
df_finfin.tail(15000).to_parquet("15k_for_testing_only_3-8.parquet", index=False)

In [249]:
df_finfin = df_finfin[:-15000]

In [251]:
df_finfin.reset_index(inplace=True, drop=True)

In [253]:
df_finfin.to_parquet("LARGE_DF_WITH_METADATA_3-8_SHUFFLED.parquet", index=False)

In [254]:
df_finfin

,text_id,output_texts,input_texts,tags,augmentation,formulae_only
0,967397,В приведенной выше схеме пусть $\mathcal{M}$ —...,В приведенной выше схеме пусть $\mathcal{M}$ —...,"[algebraic-geometry, homological-algebra]",rand_delet_and_typo,False
1,497,$\frac{z^{2}}{2}$,\frac z^ 2}} 2},[complex-analysis],rand_delet_and_typo,True
2,550867,Таких функций бесконечно много.\r\n\r\nОдним и...,Таких функций бесконечно много.\r\n\r\nОдним и...,[functions],num2word,False
3,48966,$$32\cdot\dfrac{1-\cos2x}2\cdot\dfrac{1+\cos2x...,$$32\cdot\dfrac{1-\cos2x}2\cdot\\dfrac{1+\cos2...,[trigonometry],rand_delet_and_typo,True
4,261906,"У нас есть $\sum_{i=1}^na_i\le M\,n$ для некот...",У нас есть $ сумма от итая равно 1}^na_ ай ...,"[real-analysis, sequences-and-series]",all_augments,False
...,...,...,...,...,...,...
1299988,231285,"Правило, которому должно соответствовать $\int...","Правило, которому должно соответствовать $\int...","[real-analysis, integration, improper-integrals]",rand_delet_and_typo,False
1299989,598016,"Используйте личность, которая\r\n$$\frac{d}{dx...","Используйте личность, которая\r\n$$d разделить...",[integration],all_augments,False
1299990,106290,$y- \varepsilon _2<\inf(B)=\sup(A)=L$,$ игрек отнять \epsilon с индексом 2 меньше \...,"[real-analysis, proof-verification, supremum-a...",synonyms,True
1299991,543410,я бы сказал\r\n\r\nСобытие А: «хотя бы одно» -...,я бы сказал\r\n\r\nСобытие А: «хотя бы одно» -...,"[probability-theory, probability-distributions...",synonyms,False


In [269]:
df_finfin.head(150000).to_parquet("150k_for_nums_augm_from_3-8.parquet")

In [270]:
df_finfin = df_finfin[150000:]

In [272]:
df_finfin.reset_index(drop=True,inplace=True)

In [275]:
df_finfin.to_parquet("LARGE_DF_WITH_MEDATADA_for_finetuning_from_3-8.parquet")